Some Sanity checks for our SVD

In [ ]:
import numpy as np
import pandas as pd
import re
import os
import glob
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as colors
from matplotlib.pyplot import ylabel
from matplotlib.font_manager import FontProperties

We get the dir

In [1]:
# the name of our directory
dname = os.path.dirname(os.getcwd()) + "/data_loading/data_v2_quotes/"
dname

'c:\\Users\\Jirko\\Desktop\\Hip_Lab\\analysis_scripts/data_loading/data_v2_quotes/'

Again we get our data 

In [2]:
in_out_matrix = np.load(dname + 'in_out_matrix.npy')
print(in_out_matrix.shape)

in_out_matrix_trialwise = np.load(dname + 'in_out_matrix_trialwise.npy')
print(in_out_matrix_trialwise.shape)

NameError: name 'np' is not defined